In [1]:
# IMPORTS

from bs4 import BeautifulSoup
import bs4
import requests
from selenium import webdriver

import pandas as pd
import numpy as np

### Pdf scraping

In [13]:
# USING SPACY FOR WEB SCRAPPING

driver = webdriver.Chrome("C:/chromedriver/chromedriver.exe")

C:\cygwin64\tmp\ipykernel_15976\3383127605.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/chromedriver/chromedriver.exe")


In [14]:
# LIST OF UNIQUE CATEGORIES THAT WAS NEEDED FOR THE DATASET.
# ADD OR SUBTRACT LABELS AS NEEDED

job_list = ['HR','Software Engineer', 'Data Scientist', 'Product Manager', 'Web Designer', 'Operations Manager', 'Network Security Engineer', 'Software Test Engineer', 'Java Developer', 'Business Analyst', 'DevOps Engineer']

In [15]:
# CREATING A DATAFRAME TO STORE THE LINKS OF EACH INDIVIDUAL RESUME EXAMPLE

resume_links = pd.DataFrame()
category = []
link = []

In [16]:
# LOOP TO SEARCH FOR RESUME EXAMPLES FROM THE WEBPAGE AS PER THE LABELS DEFINED ABOVE. ONLY (10*12) 120 RESUMES WILL BE COLLECTED.

# REMOVE  "&bg=85&eg=100&comp=&mod="  TO EXPAND THE SEARCH 
# (bg=85 & eg=100) FILTERS THE RESUME BASED ON SCORE OF BETWEEEN 85 AND 100, INCREASE RANGE TO EXPAND SEARCH

for job in job_list:
    JOB = job.lower()
    for i in range(1,13):   # INCREASE THE RANGE TO GET MORE RESUME DATA
        PAGE = str(i)
        URL = "https://www.livecareer.com/resume-search/search?jt=" + JOB + "&bg=85&eg=100&comp=&mod=&pg=" + PAGE
        driver.get(URL)
        aTagsInLi = driver.find_elements_by_css_selector('li a')
        for a in aTagsInLi:
            if a.get_attribute('rel') == "ugc":
                category.append(JOB)
                link.append(a.get_attribute('href'))
    

C:\cygwin64\tmp\ipykernel_15976\2431621910.py:12: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  aTagsInLi = driver.find_elements_by_css_selector('li a')


In [6]:
# STORES THE CATEGORY AND LINK TO THE RESUME WEBPAGE

resume_links["Category"] = category
resume_links["link"] = link

In [7]:
# HASHES THE LINK AND CREATES AN UNIQUE ID FOR THE ROWS

import hashlib
def id(x):
    return int(hashlib.md5(x.encode('utf-8')).hexdigest(), 16)

resume_links["id"] = resume_links["link"].apply(id)

In [8]:
# NUMBER OF RESUME COLLECTED PER DEFINED LABELS
df = resume_links
df.Category.value_counts()

hr                           132
software engineer            132
data scientist               132
product manager              132
web designer                 132
operations manager           132
network security engineer    132
software test engineer       132
java developer               132
business analyst             132
devops engineer              132
Name: Category, dtype: int64

In [9]:
df["Resume"] = ""
df["Raw_html"] = ""

In [10]:
# LOOP TO COLLECT THE RESUME DATA PRESENT IN THE LINKS AS COLECTED ABOVE
# STORED IN HTML AS WELL AS STRING FORMAT

for i in range(df.shape[0]):
    url = df.link[i]
    driver.get(url)
#     time.sleep(0.5)                  #ADDED DELAY, CAN BE REMOVED
    x = driver.page_source
    x = x.replace(">","> ")
    soup = bs4.BeautifulSoup(x, 'html.parser')
    div = soup.find("div", {"id": "document"})
    df.Raw_html[i] = div
    try:
        df.Resume[i] = div.text
    except:
#         ADD EXCEPTION IF REQUIRED
        pass



In [11]:
df.head()

,Category,link,id,Resume,Raw_html
0,hr,https://www.livecareer.com/resume-search/,20833638260276278065918687170912323191,,None
1,hr,https://www.livecareer.com/resume-search/r/tea...,10657153467684033635618204014550017025,"TEACHER, 5TH AND 6TH GRADE SCIENCE/LI...","[ , [ , [ , <div class=""name"" itemprop=""name"">..."
2,hr,https://www.livecareer.com/resume-search/r/hr-...,303012948694581463097341274704713518393,HR MANAGER Summary Human ...,"[ , [ , [ , <div class=""name thinbottomborder""..."
3,hr,https://www.livecareer.com/resume-search/r/hr-...,252695628279126685659264046008231934877,HR/PAYROLL COORDINATOR Profes...,"[ , [ , [ , <div class=""name"" itemprop=""name"">..."
4,hr,https://www.livecareer.com/resume-search/r/hr-...,162057789781141134402378787192982207371,HR MANAGER Skill Highlights ...,"[ , [ , [ , <div class=""name"" itemprop=""name"">..."


In [12]:
# STORES THE DATAFRAME

df.to_csv("Resume.csv",index=False)